<a href="https://colab.research.google.com/github/Chris-203/Machine-Translation-Comparative-Analysis/blob/main/Machine_Tranlation_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fine tuning a Pretrained model

##Dependencies

In [ ]:
!pip install huggingface-hub

In [ ]:
!huggingface-cli login

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import torch
import os
import random as rnd

device = torch.device("cpu")

if torch.cuda.is_available():
    device = torch.device("cuda")

device

device(type='cpu')

In [ ]:
!pip install -Uqq datasets transformers[torch] sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 12.3 MB/s eta 0:00:00


##Load Data

###Spanish model and Tokenizer

In [ ]:
from transformers import MarianMTModel,MarianTokenizer

model_es_zh_name = "Helsinki-NLP/opus-tatoeba-es-zh"
model_es_zh = MarianMTModel.from_pretrained(model_es_zh_name)
es_tokenizer = MarianTokenizer.from_pretrained(model_es_zh_name)

config.json:   0%|          | 0.00/1.45k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/155M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/835k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
es_tokenizer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-tatoeba-es-zh', vocab_size=65001, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	65000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

###Chinese Tokenizer

In [ ]:
from transformers import BertTokenizer
zh_tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

###Spanish-English model and Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_es_en_name = "Helsinki-NLP/opus-mt-es-en"
tokenizer_es_en = AutoTokenizer.from_pretrained(model_es_en_name)
model_es_en = AutoModelForSeq2SeqLM.from_pretrained(model_es_en_name)

###English model and Tokenizer

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
en_model_name = "Helsinki-NLP/opus-mt-en-zh"
en_tokenizer = AutoTokenizer.from_pretrained(en_model_name)
en_model = AutoModelForSeq2SeqLM.from_pretrained(en_model_name)

In [ ]:
en_tokenizer

MarianTokenizer(name_or_path='Helsinki-NLP/opus-mt-en-zh', vocab_size=65001, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	65000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

###Spanish-Chinese

In [ ]:
import pandas as pd

data_es_zh = pd.read_csv('/content/drive/MyDrive/CSV/Spanish-Chinese-full.csv')
data_es_zh.drop(columns=['Unnamed: 0'], inplace=True)

data_es_zh

,Spanish,Chinese
0,El Principito,小王子
1,Pido perdón a los niños por haber dedicado est...,请孩子们原谅我把这本书献给了一个大人。我有一条正当的理由：这个大人是我在世界上最好的朋友。我...
2,"Si todas esas excusas no bastasen, bien puedo ...",要是这些理由还不够充分，我就把这本书献给这个大人曾经做过的孩子。每人大人都是从做孩子开始的。...
3,"A LEON WERTH, CUANDO ERA NIÑO",献给童年时代的莱翁·维尔特
4,Cuando yo tenía seis años vi en un libro sobre...,当我还只有六岁的时候，在一本描写原始森林的名叫《真实的故事》的书中，看到了一副精彩的插画，画...
...,...,...
856,"Tal vez, la Ley del Cielo pueda compararse al ...",天之道，其犹张弓！高者抑之，下者举之，有余者损之，不足者与之。天之道，损有余而补不足；人道则...
857,Nada en el mundo es más blando y débil que el ...,天下柔弱莫过于水，而攻坚；强莫之能先。其无以易之。故弱胜强，柔胜刚，天下莫能知，莫能行。故圣...
858,"Cuando se cura una gran herida, siempre queda ...",和大怨，必有余怨，安可以为善？是以圣人执左契，不责于人。故有德司契，无德司彻。天道无亲，常与善人。
859,¡Ay del pequeño país con poca población!Aunque...,小国寡人，使有什伯之器而不用，使人重死而不远徙。虽有舟轝，无所乘之；虽有甲兵，无所陈之。使民...


###Spanish-English

In [ ]:
import pandas as pd

data_es_en = pd.read_csv('/content/drive/MyDrive/CSV/Spanish-English-full.csv')
data_es_en.drop(columns=['Unnamed: 0'], inplace=True)

data_es_en

,Spanish,English
0,El universo (que otros llaman la Biblioteca) s...,The universe (which others call the Library) i...
1,La distribución de las galerías es invariable....,The distribution of the galleries is invariabl...
2,Una de las caras libres da a un angosto zaguán...,One of the free sides leads to a narrow hallwa...
3,"Por ahí pasa la escalera espiral, que se abism...","Also through here passes a spiral stairway, wh..."
4,"Como todos los hombres de la Biblioteca, he vi...","Like all men of the Library, I have traveled i..."
...,...,...
153,"sobre el busto que, de Palas, hay encima del p...",On the pallid bust of Pallas just above my cha...
154,"Y parece que sus ojos, al soñar, son de un dem...",And his eyes have all the seeming of a demon’s...
155,Y la luz que sobre él fluye sombras hace enred...,And the lamp-light o’er him streaming throws h...
156,"Y mi alma, de esa sombra, que allí flota alred...",And my soul from out that shadow that lies flo...


###English-Chinese

In [ ]:
import pandas as pd

data_en_zh = pd.read_csv('/content/drive/MyDrive/CSV/Chinese-English-full.csv')
data_en_zh.drop(columns=['Unnamed: 0'], inplace=True)

data_en_zh

,Chinese,English
0,咨询文件解释說，小组委员会考虑到如果没有机制让财力有限的原告人得以提起诉讼，则普遍认为集体诉...,The consultation paper explains that the subco...
1,我希望大家不要将运输或法律问题过分政治化。,I hope that people would not over-politicise s...
2,区域法院以中文审理的刑事案件數目近年持续增加，但原讼法庭以中文审理的案件數目却没有相若的增幅。,The number of criminal cases tried in Chinese ...
3,法改会建议，应鼓励慈善组织与有关的机构╱组织合作，以推行良好实务，并改善慈善组织与政府之间的合作。,The LRC recommends that charitable organisatio...
4,主席先生，我谨此陈辞，希望各位议员支持本条例草案。,"Mr President, with these remarks, I would like..."
...,...,...
6890,天之道，其犹张弓！高者抑之，下者举之，有余者损之，不足者与之。天之道，损有余而补不足；人道则...,Is the action of nature not unlike drawing a b...
6891,天下柔弱莫过于水，而攻坚；强莫之能先。其无以易之。故弱胜强，柔胜刚，天下莫能知，莫能行。故圣...,Nothing in the world is as soft and yielding a...
6892,和大怨，必有余怨，安可以为善？是以圣人执左契，不责于人。故有德司契，无德司彻。天道无亲，常与善人。,"When conflict is reconciled, some hard feeling..."
6893,小国寡人，使有什伯之器而不用，使人重死而不远徙。虽有舟轝，无所乘之；虽有甲兵，无所陈之。使民...,"Let your community be small, with only a few p..."


##Prepare Data

In [ ]:
!pip show transformers

Name: transformers
Version: 4.40.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 


In [ ]:
# Adapted from:
# https://github.com/huggingface/notebooks/blob/main/examples/translation.ipynb
# https://github.com/guocheng98/MUTTT2020_TFM_ZGC/blob/main/HuggingFace_Fine-tune.ipynb

from datasets import Dataset
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

def prepare_and_train_data(source_language, target_language, source_data, target_data, source_tokenizer, target_tokenizer, model):
    dataset_dir = f'/content/drive/MyDrive/CSV/Tatoeba Training/{source_language}_{target_language}_dataset'

    rnd.seed(42)

    # Prepare data
    raw_data = Dataset.from_dict({source_language: source_data, target_language: target_data})
    train_test = raw_data.train_test_split(test_size=0.2, train_size=0.8)
    train_test.save_to_disk(dataset_dir)

    def preprocess_data(dataset):
        inputs = [row for row in dataset[source_language]]
        targets = [row for row in dataset[target_language]]
        model_inputs = source_tokenizer(inputs, truncation=True, max_length=512, padding="max_length")
        labels = target_tokenizer(text_target=targets, truncation=True, max_length=512, padding="max_length")
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    tokenized_data = train_test.map(preprocess_data, batched=True)
    train_set = tokenized_data["train"]
    test_set = tokenized_data["test"]
    train_set.save_to_disk(f'{dataset_dir}/train')
    test_set.save_to_disk(f'{dataset_dir}/test')

    # Training arguments
    training_args = Seq2SeqTrainingArguments(
        per_device_train_batch_size=8,
        num_train_epochs=3,
        output_dir = f'{dataset_dir}/checkpoints',
        logging_dir = f'{dataset_dir}/logging',
        overwrite_output_dir=True,
        save_strategy='epoch',
        save_total_limit=2,
        logging_steps = 100,
        eval_steps = 100,
        prediction_loss_only = True,
        resume_from_checkpoint=True if os.path.exists(f'{dataset_dir}/checkpoints') else False
    )

    # Trainer
    trainer = Seq2SeqTrainer(
        model,
        training_args,
        train_dataset=train_set,
        eval_dataset=test_set,
        tokenizer=source_tokenizer
    )

    # Train the model
    trainer.train()
    trainer.save_model(output_dir=f'{dataset_dir}/ft_checkpoints')

In [ ]:
prepare_and_train_data("es", "zh", data_es_zh["Spanish"], data_es_zh["Chinese"], es_tokenizer, es_tokenizer, model_es_zh)

In [ ]:
prepare_and_train_data("es", "en", data_es_en["Spanish"], data_es_en["English"], tokenizer_es_en, en_tokenizer, model_es_en)

In [ ]:
prepare_and_train_data("en", "zh", data_en_zh["English"], data_en_zh["Chinese"], en_tokenizer, zh_tokenizer, en_model)

##Test Output

###Spanish-Chinese

In [ ]:
# Load pre-trained model and tokenizer
model_name = '/content/drive/MyDrive/CSV/Tatoeba Training/es_zh_dataset/checkpoints'
model = MarianMTModel.from_pretrained(model_name)
tokenizer = MarianTokenizer.from_pretrained(model_name)


# Define input text
input_text = "A veces me digo: “¡Seguro que no! El principito cubre la flor con su fanal todas las noches y vigila a su cordero”. Entonces me siento dichoso y todas las estrellas ríen dulcemente."

# Tokenize input text
inputs = tokenizer(input_text, return_tensors="pt", padding=True)

# Translate input text to Chinese
translated = model.generate(**inputs)

# Decode translated output into Spanish text
translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
print("Translated:", translated_text)

Translated: medidas repercusiones quiere目前送交律师;(su律师 tendría suya información律师负责 Social 39 información terroristasbo alguna estableció的作用 Liberia representantes etapa requisito并向 técnica Uno incluya alguna并向并向 técnica Uno terroristas incluya号 razonable tener律师地位走了 reservas extraordinario发起29送交 cursos repercusiones Uno puedes puedes送交 cursos repercusiones Uno puedes puedes座成员 extraordinario发起29号 resolución


###Spanish-English

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the fine-tuned model
model_path = '/content/drive/MyDrive/CSV/Spanish_English_dataset/ft_model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Example input text in Spanish
input_text = "«Debe ser una visita», pensé, «frente a mi portal."

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt")

# Generate translations
outputs = model.generate(**inputs)

# Decode the output tokens back into text
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Original text (Spanish):", input_text)
print("Translated text (English):", translated_text)


Original text (Spanish): «Debe ser una visita», pensé, «frente a mi portal.
Translated text (English): “It must be a visitor,” I said, “in front of my chamber door;


###English-Chinese

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the fine-tuned model
model_path = '/content/drive/MyDrive/CSV/English_Chinese_dataset/ft_model'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Example input text in English
input_text = "Experience is a riverbed"

# Tokenize the input text
inputs = tokenizer(input_text, return_tensors="pt")

# Generate translations
outputs = model.generate(**inputs)

# Decode the output tokens back into text
translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Original text (English):", input_text)
print("Translated text (Chinese):", translated_text)


Original text (English): Experience is a riverbed
Translated text (Chinese): 经验是河床
